### Silver Layer

In [ ]:
import os
import sys

# current_dir = os.getcwd()
# project_root = os.path.abspath(os.path.join(current_dir,"..","..",".."))

# # print(project_root)
# # print(current_dir)

# sys.path.append(project_root)

In [ ]:
def widget(name, default=None):
    try:
        return dbutils.widgets.get(name)
    except:
        return default



dbutils.widgets.text("pipeline_id", "")
dbutils.widgets.text("run_id", "")
dbutils.widgets.text("task_id", "")
dbutils.widgets.text("processed_timestamp", "")
dbutils.widgets.text("catalog", "")



pipleline_id = widget("pipeline_id")
run_id = widget("run_id")
task_id = widget("task_id")
processed_timestamp = widget("processed_timestamp")
catalog = widget("catalog")

In [ ]:
from utils.citibike_utils import get_trip_duration_mins
from utils.datetime_utils import timestamp_to_date_col
from pyspark.sql.functions import create_map, lit

In [ ]:
df_silver = spark.read.table(f"{catalog}.01_bronze.jc_citibike")

In [8]:
df_silver = get_trip_duration_mins(spark, df_silver, "started_at","ended_at","trip_duration_mins")

In [9]:
df_silver = timestamp_to_date_col(spark, df_silver, "started_at", "trip_start_date")

In [10]:
df_silver = df_silver.withColumn("metadata",
                    create_map(
                        lit("pipeline_id"), lit(pipleline_id),
                        lit("run_id"), lit(run_id),
                        lit("task_id"), lit(task_id),
                        lit("processed_timestamp"), lit(processed_timestamp)
                    ))

In [ ]:
df_silver = df_silver.select(
    "ride_id",
    "trip_start_date",
    "started_at",
    "ended_at",
    "start_station_name",
    "end_station_name",
    "trip_duration_mins",
    "metadata"
)

In [ ]:
(df_silver
    .write
    .mode("overwrite")
    .option("overwriteSchema","true")
    .saveAsTable(f"{catalog}.02_silver.jc_citibike")
    
)